In [ ]:
pip install azure-storage-blob


In [4]:
from azure.storage.blob import BlobServiceClient
import os


# Azure storage connection string
connection_string = "DefaultEndpointsProtocol=https;AccountName=sapocopenai;AccountKey=H30mVmhmyKpFmLUkFSKpDO3CUe+jbcG8aGZ8TgCRNTQj5Ac5HB+649BzYypyo9eW0W9BRy9Z0oFr+ASt6hAVYw==;EndpointSuffix=core.windows.net"
container_name = "bulk-upload-demo"
local_folder = "Sharepoint files download/file_uploads_bkp"

# Create a BlobServiceClient
blob_service_client = BlobServiceClient.from_connection_string(connection_string)

# Get the container client
container_client = blob_service_client.get_container_client(container_name)

# Upload files
for root, dirs, files in os.walk(local_folder):
    for file in files:
        blob_client = container_client.get_blob_client(os.path.join(root, file))
        with open(os.path.join(root, file), "rb") as data:
            blob_client.upload_blob(data)
        print(f"Uploaded {file} to container {container_name}")

Uploaded aaaa.pdf to container bulk-upload-demo
Uploaded Advanced Analytics - from AI to Z - eBook - SDG Group.pdf to container bulk-upload-demo
Uploaded Best of ASCO Deck 2024 Leave Behind - Copy.pdf to container bulk-upload-demo
Uploaded Best of ASCO Deck 2024 Leave Behind.pdf to container bulk-upload-demo
Uploaded Daiichi Sankyo_ DS8201-801_RoutineSponsorMeetingMinutes_02Oct2024_Final.pdf to container bulk-upload-demo
Uploaded Data Anonymization.txt to container bulk-upload-demo
Uploaded DSI_Application_Guide_HelpDesk_KUMO AI.docx to container bulk-upload-demo
Uploaded DSI_SOP_KUMO AI.docx to container bulk-upload-demo
Uploaded DSI_User_mannual_Kumo-AI 1.pdf to container bulk-upload-demo
Uploaded EDA video script_new.pptx to container bulk-upload-demo
Uploaded FAQs About Clinical Studies - Copy.docx to container bulk-upload-demo
Uploaded FAQs About Clinical Studies - Part2 1.docx to container bulk-upload-demo
Uploaded FAQs About Clinical Studies - Part2.docx to container bulk-upload

In [5]:
from azure.core.credentials import AzureKeyCredential
from azure.search.documents.indexes import SearchIndexClient
from azure.search.documents.indexes.models import SimpleField, SearchIndex, SearchableField, SearchField, SearchFieldDataType

# Azure Search credentials
search_service_name = "poc-openai-cogsrch"

admin_key = AzureKeyCredential("TBpoXA6at3L38HsBWSRHnZWfstC07Txt7kDulY7ccDAzSeAHslm9")
index_name = "ssirish_bulk"
# indexer_client = SearchIndexerClient(endpoint=f"https://{search_service_name}.search.windows.net", credential=admin_key)

# admin_key = "your_admin_key"
# index_name = "your_index_name"

# Initialize SearchIndexClient
search_service_endpoint = f"https://{search_service_name}.search.windows.net"
search_client = SearchIndexClient(endpoint=search_service_endpoint, credential=admin_key)

# Define the search index schema
fields = [
    SimpleField(name="id", type="Edm.String", key=True),
    SearchableField(name="content", type="Edm.String", searchable=True, filterable=False, sortable=False),
    SearchField(name="metadata_storage_path", type=SearchFieldDataType.String)
]

index = SearchIndex(name=index_name, fields=fields)

# Create the search index
search_client.create_index(index)
print(f"Index '{index_name}' created.")


Index 'ssirish_bulk' created.


In [9]:
from langchain.document_loaders import UnstructuredPDFLoader, PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

# path = "2_sample_biology.pdf"
path = "C:/Users/ssirish/Downloads/Sharepoint files download/file_uploads_bkp/response(10)_Aditi.pdf"
loader = PyPDFLoader(path)

doc = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap  = 200)
docs = text_splitter.split_documents(doc)

In [ ]:
# docs

[Document(page_content='The document is a financial forecasting deck that includes various elements such as:\n1. **Table with Financial Forecast Details**:\n   - **Header Information**:\n     - **Milestone Owner**: Richard Park\n     - **Email**: rpark@dsi.com\n     - **Last updated**: 09/09/2024\n   - **Table Columns**:\n     1. Brand\n     2. Forecast Type\n     3. Financial Forecast\n     4. Geography\n     5. Date (mm/dd/yyyy)\n     6. Status\n     7. Key Assumptions and/or Dependencies (300 Character Max)\n2. **Project Timeline for 2024**:\n   - Divided into quarters (Qtr 1, Qtr 2, Qtr 3, Qtr 4) and months.\n   - Includes various projects and milestones under different categories:\n     - **Financial Forecasts**\n     - **DATO-DXd** (with specific projects like TB-01, TB-02, etc.)\n     - **ENHERTU** (with specific projects like DB-05, DB-06, etc.)\n3. **Iconography**:\n   - A green icon with a white dollar sign ($) inside a house-like shape, representing\nfinancial or monetary as

In [11]:
from azure.search.documents import SearchClient
from azure.core.credentials import AzureKeyCredential

admin_key = AzureKeyCredential("TBpoXA6at3L38HsBWSRHnZWfstC07Txt7kDulY7ccDAzSeAHslm9")
# index_name = "bulk-upload-testing"
index_name = "ssirish_bulk"
search_service_name = "poc-openai-cogsrch"
search_service_endpoint = f"https://{search_service_name}.search.windows.net"

# Initialize SearchClient
index_client = SearchClient(endpoint=search_service_endpoint, index_name=index_name, credential=admin_key)



In [12]:
documents = []
for i, k in enumerate(docs):
    document = {
        "id": str(i+1),
        "content": k.page_content,
        "metadata_storage_path": k.metadata['source']
    }
    documents.append(document)

In [13]:
index_client.upload_documents(documents=documents)
print("Documents uploaded to the index.")

Documents uploaded to the index.


In [16]:
results = index_client.search("Clear chat",search_fields=["content"], top=5)

In [17]:
for result in results:
    print(result)